In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip -q install --upgrade ollama

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%                                                                    12.8%#######################################                                         58.7%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
import os
get_ipython().system = os.system

In [5]:
!ollama serve &

0

In [6]:
!ollama pull gemma2

0

In [7]:
import json
from tqdm import tqdm
import ollama

# from langchain_community.chat_models import ChatOllama
# from langchain.document_loaders import WikipediaLoader
# # from langchain_community.llms import Ollama
# from langchain_ollama import OllamaLLM
# from langchain.chains import LLMChain
# from langchain.prompts.chat import (ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate)
# from langchain import PromptTemplate

In [8]:
# Path to your input and output files
input_file = "/kaggle/input/task-2-ea-mt-entity-aware-machine-translation/semeval.test_hidden/test_without_targets/ja_JP.jsonl"
output_file = "translated_data_en_ja_gemma2-v2.jsonl"

In [9]:
def translate_text(obj):
    prompt = f"""
    Translate the following English text into Japanese with more attention on accurately translating the given named entities. 
    Do not add extra text.
    
    Entity Types: {', '.join(obj['entity_types'])}
    Text: "{obj['source']}"
    
    Output the result in JSON format:
    {{
        "id": "{obj['id']}",
        "predicted_text": "<translated_text>"
    }}
    """
    
    response = ollama.chat(
        model="gemma2",
        messages=[{"role": "system", "content": "You are a translation assistant."},
                  {"role": "user", "content": prompt}],
        stream=False
    )
    
    # Extract generated text and parse JSON
    try:
        content = response.message.content.strip()
        # print(response.message.content)
        # Remove markdown code block (```json ... ```)
        if content.startswith("```json"):
            content = content[7:-3].strip()  # Remove first 7 (````json\n`) and last 3 characters (`\n````)
        
        output_json = json.loads(content)
        return output_json
    except json.JSONDecodeError:
        print(f"Error parsing response for ID: {obj['id']}")
        return None

In [10]:
with open(input_file, "r", encoding="utf-8") as f:
    total_lines = sum(1 for _ in f)

In [11]:
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
     with tqdm(total=total_lines, desc="Generating Predictions", colour="blue") as pbar:
        for line in infile:
            obj = json.loads(line.strip())
            translated_obj = translate_text(obj)
            # print(translated_obj)
            # break
            if translated_obj and ("predicted_text" in translated_obj):
                translated_obj = {
                    "id": obj["id"],
                    "source_language": "English",
                    "target_language": "Japanese", 
                    "text": obj["source"],
                    "prediction": translated_obj["predicted_text"]
                }
                outfile.write(json.dumps(translated_obj, ensure_ascii=False) + "\n")
            pbar.update(1)

print(f"Translation completed! Output saved to {output_file}")

Generating Predictions:   0%|          | 5/5108 [00:18<5:40:38,  4.01s/it]

Error parsing response for ID: b055e67d0654b71d


Generating Predictions:   1%|          | 32/5108 [01:09<2:49:18,  2.00s/it]

Error parsing response for ID: c3951b0d7cc95eca


Generating Predictions:   1%|          | 33/5108 [01:14<3:48:47,  2.70s/it]

...

Error parsing response for ID: 6019fc32da1aa0da


Generating Predictions:  99%|█████████▉| 5066/5108 [3:24:18<02:34,  3.67s/it]

Error parsing response for ID: ac0a58fcc5a47e60


Generating Predictions: 100%|██████████| 5108/5108 [3:25:53<00:00,  2.42s/it]

Translation completed! Output saved to translated_data_en_ja_gemma2-v2.jsonl
